# Environment Setup

In [ ]:
#!pip install gym_super_mario_bros
#!pip install stable_baselines3

In [4]:
# Import mario
import gym_super_mario_bros
# Import environment base class for a wrapper 
from gym import Env 
# Import the space shapes for the environment
from gym.spaces import Box
# Import simpler movements
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
# Import to reduce number of movments
from nes_py.wrappers import JoypadSpace
# Import numpy to calculate frame delta 
import numpy as np
# Import opencv for grayscaling
import cv2

In [13]:
# Creating custom environment 
class Mario(Env): 
    
    def __init__(self):
        super().__init__()
        # Specify observation space 
        self.observation_space = Box(low=0, high=255, shape=(100, 100, 1), dtype=np.uint8)
        # Startup and instance of the game
        self.game = gym_super_mario_bros.make("SuperMarioBros-v0")
        
        
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        
        # ***Reward shaping***       
        coins, flag_get, life, world = info["coins"], info["flag_get"], int(info["life"]), info["world"]
        status, y_pos, x_pos, score  = info["status"], info["y_pos"], info["x_pos"], info["score"]
        
        # Calculate reward deltas
        coins_delta = coins - self.coins
        self.coins = coins
        score_delta = score - self.score
        self.score = score
        moving_forward_delta = x_pos - self.x_pos
        self.x_pos = x_pos

        reward = score_delta*0.5 + moving_forward_delta*0.01 - self.time_ticking*0.02
        
        # Jumped over a hole
        #if np.any(obs[97:98,32:33] == 167):
        #    reward += 0.009
        
        return obs, reward, done, info
    
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create game variables
        self.coins = 0
        self.flag_get = False
        self.score = 0
        self.x_pos = 40
        self.time_ticking = 1
        self.y_pos = 91
        
        return obs
    
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (100,100), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (100,100,1))
        return channels 
    
    
    def render(self, *args, **kwargs):
        self.game.render()
        
        
    def close(self):
        self.game.close()

# Testing Environment

In [14]:
"""
# Game with random actions

# Importing sleep to slow down game
from time import sleep

# Creating environment
env = Mario()
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Checking if env is correct
from stable_baselines3.common import env_checker
print("If None is returned\n the env is set correctly\n", env_checker.check_env(env))


obs = env.reset()
result = 0 

for i in range(300):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    result += reward
    env.render()
    sleep(0.006)
    i += 1
    if i % 100 == 0:
        result += reward
        print(result)
        
"""

TypeError: __init__() got an unexpected keyword argument 'stages'

In [ ]:
"""  
# Displaying preprocessed frames
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import matplotlib.pyplot as plt

env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

plt.figure(figsize=(16,9))

# Skipping some first black frames
for _ in range(6):
    obs, reward, done, info = env.step([env.action_space.sample()])
# Displaying 4 frames as movement
for i in range(obs.shape[3]):
    obs, reward, done, info = env.step([env.action_space.sample()])
    plt.subplot(1,4,i+1)
    plt.imshow(obs[0][:,:,i])
plt.show();
"""